#CÂU 1

In [2]:
import sqlite3
import pandas as pd

# Tạo dữ liệu mẫu
data = [
    (8, 9),
    (7.5, 8.5),
    (6, 7),
    (7, 6)
]

# Tạo kết nối SQLite in-memory
conn = sqlite3.connect(':memory:')
cur = conn.cursor()

# Tạo bảng
cur.execute('CREATE TABLE Scores (A REAL, B REAL)')
cur.executemany('INSERT INTO Scores (A, B) VALUES (?, ?)', data)

# Truy vấn tính hệ số tương quan Pearson
query = """
SELECT
    (COUNT(*) * SUM(A * B) - SUM(A) * SUM(B)) /
    (SQRT(COUNT(*) * SUM(A * A) - SUM(A) * SUM(A)) *
     SQRT(COUNT(*) * SUM(B * B) - SUM(B) * SUM(B))) AS r_AB
FROM Scores
"""

cur.execute(query)
result = cur.fetchone()[0]
print("Hệ số tương quan Pearson r_AB =", result)


Hệ số tương quan Pearson r_AB = 0.6910506641398757


#CÂU 2


In [1]:
import sqlite3
import pandas as pd
from scipy.stats import chi2_contingency

# Kết nối SQLite trong bộ nhớ
conn = sqlite3.connect(':memory:')
cursor = conn.cursor()

# 1. Tạo bảng và thêm dữ liệu
cursor.execute('''
    CREATE TABLE ratings (
        day TEXT,
        car_model TEXT,
        score REAL
    )
''')

ratings_data = [
    ('Day 1', 'A', 8),
    ('Day 1', 'B', 9),
    ('Day 1', 'C', 7),
    ('Day 2', 'A', 7.5),
    ('Day 2', 'B', 8.5),
    ('Day 2', 'C', 7),
    ('Day 3', 'A', 6),
    ('Day 3', 'B', 7),
    ('Day 3', 'C', 8),
    ('Day 4', 'A', 7),
    ('Day 4', 'B', 6),
    ('Day 4', 'C', 5),
]
cursor.executemany('INSERT INTO ratings (day, car_model, score) VALUES (?, ?, ?)', ratings_data)
conn.commit()

# 2. Truy vấn dữ liệu và làm tròn điểm
query = '''
SELECT car_model, day, ROUND(score) AS rounded_score
FROM ratings
'''
df = pd.read_sql_query(query, conn)

# 3. Kiểm định χ² theo mẫu xe (car_model)
table_model = pd.crosstab(df['car_model'], df['rounded_score'])
print("Bảng chéo (car_model vs rounded_score):\n", table_model)

chi2_model, p_model, dof_model, expected_model = chi2_contingency(table_model)
print(f"\n[Kiểm định theo mẫu xe] Chi-squared = {chi2_model:.4f}, p = {p_model:.4f}")
if p_model < 0.05:
    print("Kết luận: Có sự khác biệt đáng kể giữa các mẫu xe.")
else:
    print("Kết luận: Không có sự khác biệt đáng kể giữa các mẫu xe.")

# 4. Kiểm định χ² theo ngày (day)
table_day = pd.crosstab(df['day'], df['rounded_score'])
print("\nBảng chéo (day vs rounded_score):\n", table_day)

chi2_day, p_day, dof_day, expected_day = chi2_contingency(table_day)
print(f"\n[Kiểm định theo ngày] Chi-squared = {chi2_day:.4f}, p = {p_day:.4f}")
if p_day < 0.05:
    print("Kết luận: Kết quả phụ thuộc vào ngày kiểm tra.")
else:
    print("Kết luận: Kết quả không phụ thuộc vào ngày kiểm tra.")


Bảng chéo (car_model vs rounded_score):
 rounded_score  5.0  6.0  7.0  8.0  9.0
car_model                             
A                0    1    1    2    0
B                0    1    1    0    2
C                1    0    2    1    0

[Kiểm định theo mẫu xe] Chi-squared = 9.5000, p = 0.3019
Kết luận: Không có sự khác biệt đáng kể giữa các mẫu xe.

Bảng chéo (day vs rounded_score):
 rounded_score  5.0  6.0  7.0  8.0  9.0
day                                   
Day 1            0    0    1    1    1
Day 2            0    0    1    1    1
Day 3            0    1    1    1    0
Day 4            1    1    1    0    0

[Kiểm định theo ngày] Chi-squared = 8.0000, p = 0.7851
Kết luận: Kết quả không phụ thuộc vào ngày kiểm tra.


#CÂU 3

In [3]:
import sqlite3
import pandas as pd

# Tạo database giả lập
conn = sqlite3.connect(':memory:')
cursor = conn.cursor()

# Tạo bảng flights
cursor.execute('''
    CREATE TABLE flights (
        flight_id INTEGER PRIMARY KEY,
        departure_time INTEGER
    )
''')

# Dữ liệu ví dụ
flights_data = [
    (1, 830),
    (2, 1445),
    (3, 1015),
    (4, 50),      # 0:50 AM
]
cursor.executemany('INSERT INTO flights VALUES (?, ?)', flights_data)
conn.commit()

# Truy vấn + chuyển đổi thời gian bằng Python
df = pd.read_sql_query('SELECT * FROM flights', conn)

# Chuyển đổi sang định dạng HH:MM
def int_to_time(t):
    t = int(t)
    hour = t // 100
    minute = t % 100
    return f"{hour:02d}:{minute:02d}"

df['formatted_time'] = df['departure_time'].apply(int_to_time)
print(df)


   flight_id  departure_time formatted_time
0          1             830          08:30
1          2            1445          14:45
2          3            1015          10:15
3          4              50          00:50


#CÂU 4

In [4]:
import numpy as np

# Tạo bảng dữ liệu
cursor.execute('''
    CREATE TABLE data (
        id INTEGER PRIMARY KEY,
        value REAL
    )
''')

data_values = [(1,), (2,), (3,), (100,), (4,), (5,), (6,)]
cursor.executemany('INSERT INTO data (value) VALUES (?)', data_values)
conn.commit()

# Lấy dữ liệu
df_data = pd.read_sql_query('SELECT * FROM data', conn)
values = df_data['value'].values

# Tính median và MAD
median = np.median(values)
mad = np.median(np.abs(values - median))

# Tìm ngoại lệ
threshold = 1.5 * mad
df_data['outlier'] = np.abs(values - median) > threshold
print(df_data)


   id  value  outlier
0   1    1.0    False
1   2    2.0    False
2   3    3.0    False
3   4  100.0     True
4   5    4.0    False
5   6    5.0    False
6   7    6.0    False


#CÂU 5

In [5]:
import sqlite3
import pandas as pd

# Tạo dữ liệu mẫu
patients = [
    ("Nguyen", 60, 170),
    ("Nguyen", 61, 169),
    ("Tran", 55, 160),
    ("Le", 60, 170),
    ("Nguyen", 59, 171),
]

# Kết nối SQLite
conn = sqlite3.connect(':memory:')
cur = conn.cursor()

# Tạo bảng và thêm dữ liệu
cur.execute("CREATE TABLE Patient (last_name TEXT, weight REAL, height REAL)")
cur.executemany("INSERT INTO Patient VALUES (?, ?, ?)", patients)

# Truy vấn kiểm tra các cặp người có khả năng là cùng 1 người
query = """
SELECT
    a.last_name AS name1, a.weight AS weight1, a.height AS height1,
    b.last_name AS name2, b.weight AS weight2, b.height AS height2
FROM Patient a
JOIN Patient b
  ON a.rowid < b.rowid
WHERE
    a.last_name = b.last_name
    AND ABS(a.weight - b.weight) <= 2
"""

df_similar = pd.read_sql_query(query, conn)
print(df_similar)


    name1  weight1  height1   name2  weight2  height2
0  Nguyen     60.0    170.0  Nguyen     59.0    171.0
1  Nguyen     60.0    170.0  Nguyen     61.0    169.0
2  Nguyen     61.0    169.0  Nguyen     59.0    171.0
